<a href="https://colab.research.google.com/github/JoeMiddleton14/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy


#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [4]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'Education', 'DistanceFromHome', 'JobSatisfaction', 'OverTime',
                    'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]


# Show the data types for X_df
X_df.dtypes


Age                         int64
Education                   int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_df.loc[:, 'OverTime'] = X_df['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)

X_df = X_df.apply(pd.to_numeric)

X_df.dtypes

Age                        int64
Education                  int64
DistanceFromHome           int64
JobSatisfaction            int64
OverTime                   int64
StockOptionLevel           int64
WorkLifeBalance            int64
YearsAtCompany             int64
YearsSinceLastPromotion    int64
NumCompaniesWorked         int64
dtype: object

In [7]:
# Convert the 'OverTime' column to numeric in X_train and X_test to ensure consistency
X_train['OverTime'] = X_train['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)
X_test['OverTime'] = X_test['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)


In [8]:
# Create a StandardScaler
scaler = StandardScaler()


# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [9]:

# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse=False)


# Fit the encoder to the training data
dept_encoder.fit(y_train[['Department']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept_encoded = dept_encoder.transform(y_train[['Department']])
y_test_dept_encoded = dept_encoder.transform(y_test[['Department']])




/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse=False)


# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_attr_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_test_attr_encoded = attrition_encoder.transform(y_test[['Attrition']])



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Create, Compile, and Train the Model

In [11]:
# Find the number of columns in the X training data
input_dim = X_train_scaled.shape[1]
print("Number of input features:", input_dim)

# Create the input layer
inputs = layers.Input(shape=(input_dim,))


# Create at least two shared layers
shared_layer_1 = layers.Dense(64, activation='relu')(inputs)
shared_layer_2 = layers.Dense(32, activation='relu')(shared_layer_1)


Number of input features: 10


In [12]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_branch = layers.Dense(16, activation='relu')(shared_layer_2)


# Create the output layer
department_output = layers.Dense(y_train_dept_encoded.shape[1], activation='softmax', name='department_output')(department_branch)



In [13]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_branch = layers.Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
attrition_output = layers.Dense(y_train_attr_encoded.shape[1], activation='softmax', name='attrition_output')(attrition_branch)



In [14]:
# Create the model
model = Model(inputs=inputs, outputs=[department_output, attrition_output])


# Compile the model
model.compile(optimizer=Adam(),
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'categorical_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})


# Summarize the model
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 64)                   704       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 32)                   2080      ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 16)                   528       ['dense_1[0][0]']             
                                                                                              

In [15]:
# Train the model
history = model.fit(X_train_scaled,
                    {'department_output': y_train_dept_encoded, 'attrition_output': y_train_attr_encoded},
                    epochs=20,
                    batch_size=32,
                    validation_split=0)


Epoch 1/20
37/37 [==============================] - 2s 2ms/step - loss: 1.4910 - department_output_loss: 0.9517 - attrition_output_loss: 0.5393 - department_output_accuracy: 0.6471 - attrition_output_accuracy: 0.7806
Epoch 2/20
37/37 [==============================] - 0s 3ms/step - loss: 1.2277 - department_output_loss: 0.8013 - attrition_output_loss: 0.4264 - department_output_accuracy: 0.6505 - attrition_output_accuracy: 0.8316
Epoch 3/20
37/37 [==============================] - 0s 2ms/step - loss: 1.1668 - department_output_loss: 0.7714 - attrition_output_loss: 0.3955 - department_output_accuracy: 0.6505 - attrition_output_accuracy: 0.8384
Epoch 4/20
37/37 [==============================] - 0s 3ms/step - loss: 1.1431 - department_output_loss: 0.7662 - attrition_output_loss: 0.3769 - department_output_accuracy: 0.6522 - attrition_output_accuracy: 0.8469
Epoch 5/20
37/37 [==============================] - 0s 2ms/step - loss: 1.1251 - department_output_loss: 0.7600 - attrition_output_l

In [17]:
# Evaluate the model with the testing data
eval_results = model.evaluate(X_test_scaled,
                              {'department_output': y_test_dept_encoded, 'attrition_output': y_test_attr_encoded})


10/10 [==============================] - 0s 6ms/step - loss: 1.1940 - department_output_loss: 0.8183 - attrition_output_loss: 0.3757 - department_output_accuracy: 0.6599 - attrition_output_accuracy: 0.8741


In [18]:
# Print the accuracy for both department and attrition
# Print the accuracy for both department and attrition
print("Department accuracy:", eval_results[3])  # Accuracy for department_output
print("Attrition accuracy:", eval_results[4])   # Accuracy for attrition_output

Department accuracy: 0.6598639488220215
Attrition accuracy: 0.8741496801376343


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.